In [72]:
import pandas as pd
import numpy as np
from scalecast.Forecaster import Forecaster
from scalecast.auxmodels import auto_arima
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
sns.set(rc={'figure.figsize':(14,7)})

In [73]:
df = pd.read_csv('Corptax_all.csv')
df[['Date Closed','Date Created']] = df[['Date Closed','Date Created']].apply(pd.to_datetime, errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190892 entries, 0 to 190891
Data columns (total 20 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Unnamed: 0         190892 non-null  int64         
 1   index              190892 non-null  int64         
 2   Case Number        190892 non-null  int64         
 3   Subject            190891 non-null  object        
 4   Date Created       190892 non-null  datetime64[ns]
 5   Date Closed        190772 non-null  datetime64[ns]
 6   Origin             190892 non-null  object        
 7   Priority           190892 non-null  object        
 8   Team               190890 non-null  object        
 9   Case Type          187269 non-null  object        
 10  SLA                190843 non-null  object        
 11  Problem Statement  187361 non-null  object        
 12  Product            190765 non-null  object        
 13  Product Area       190672 non-null  object  

In [74]:
fill_dt = pd.Timestamp.now().normalize()
df['Date Closed'] = df['Date Closed'].fillna(fill_dt)

In [75]:
df.isna().sum()

Unnamed: 0                0
index                     0
Case Number               0
Subject                   1
Date Created              0
Date Closed               0
Origin                    0
Priority                  0
Team                      2
Case Type              3623
SLA                      49
Problem Statement      3531
Product                 127
Product Area            220
Product Category     143167
Product Type         185713
Jurisdiction          67731
Resolution             3119
KB #                 138592
KB Title             140655
dtype: int64

In [76]:
df = df.dropna(axis=0, subset=['Product', 'Product Area'])

In [77]:
df.isna().sum()

Unnamed: 0                0
index                     0
Case Number               0
Subject                   1
Date Created              0
Date Closed               0
Origin                    0
Priority                  0
Team                      2
Case Type              3610
SLA                      47
Problem Statement      3522
Product                   0
Product Area              0
Product Category     142947
Product Type         185493
Jurisdiction          67618
Resolution             3109
KB #                 138420
KB Title             140483
dtype: int64

In [78]:
df['Year']= df['Date Created'].dt.year
df['Month']= df['Date Created'].dt.month
df['Week']= df['Date Created'].dt.isocalendar().week
df['week_num'] = df['Week'] + (df['Year'] - df['Year'].min())*52

In [79]:
df['Date'] = df['Date Created'].dt.date 
drop_columns = ['Date Closed','Date Created','Unnamed: 0',]
df.drop(drop_columns, axis=1, inplace=True)
df.head()

,index,Case Number,Subject,Origin,Priority,Team,Case Type,SLA,Problem Statement,Product,...,Product Type,Jurisdiction,Resolution,KB #,KB Title,Year,Month,Week,week_num,Date
0,0,204977,Efile Acceptance,Web,Medium,Tax Preparation,Efile,Platinum,We transmitted a Michigan return still have no...,Compliance,...,NaN,US.MI,Answer exists KB,041324813494120,How to understand the Transmission Monitor,2017,1,1,1,2017-01-02
1,1,204979,Oregon STAT help: Previous Case 203787 Client ...,Web,High,Tax Preparation,Question / How To,Silver,Oregon STAT help: Previous Case 203787 Client ...,Provision,...,NaN,US.OR,Customer no longer needs assistance,NaN,NaN,2017,1,1,1,2017-01-02
2,2,204980,Data map incorrectly pulling in values,Web,Medium,Tax Preparation,Question / How To,Silver,Corptax Office Import - Credit Account imports...,Office - Excel Add-In,...,NaN,US,Answer exists KB,041414821333274,Corptax® Office - How to place an IF formula i...,2017,1,1,1,2017-01-02
3,3,204981,The New Corptax Connect Website is not properl...,Web,High,Tax Preparation,Problem / Error,Silver,Searches can't be performed on the new system....,Websites,...,NaN,NaN,Answer exists KB,161216104028927,Connections Error - Uh Oh something went wron...,2017,1,1,1,2017-01-02
4,4,204986,LD400 is not reflecting a temporary adjustment,E-mail,Medium,Tax Preparation,Problem / Error,Platinum,LD400 is not reflecting a temporary adjustment,Compliance,...,NaN,US,Answer exists KB,041622907012748,Adjustments do not flow to LD400 workpaper and...,2017,1,1,1,2017-01-03


In [80]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')

In [81]:
df.rename(columns={'Case Number':'Case_Number', 'Product Area': 'Product_Area'}, inplace=True)

In [82]:
df['PPA'] = df['Product'] + " : " + df['Product_Area']
df.head()

,index,Case_Number,Subject,Origin,Priority,Team,Case Type,SLA,Problem Statement,Product,...,Jurisdiction,Resolution,KB #,KB Title,Year,Month,Week,week_num,Date,PPA
0,0,204977,Efile Acceptance,Web,Medium,Tax Preparation,Efile,Platinum,We transmitted a Michigan return still have no...,Compliance,...,US.MI,Answer exists KB,041324813494120,How to understand the Transmission Monitor,2017,1,1,1,2017-01-02,Compliance : Returns / Reports
1,1,204979,Oregon STAT help: Previous Case 203787 Client ...,Web,High,Tax Preparation,Question / How To,Silver,Oregon STAT help: Previous Case 203787 Client ...,Provision,...,US.OR,Customer no longer needs assistance,NaN,NaN,2017,1,1,1,2017-01-02,Provision : State
2,2,204980,Data map incorrectly pulling in values,Web,Medium,Tax Preparation,Question / How To,Silver,Corptax Office Import - Credit Account imports...,Office - Excel Add-In,...,US,Answer exists KB,041414821333274,Corptax® Office - How to place an IF formula i...,2017,1,1,1,2017-01-02,Office - Excel Add-In : Import / Export
3,3,204981,The New Corptax Connect Website is not properl...,Web,High,Tax Preparation,Problem / Error,Silver,Searches can't be performed on the new system....,Websites,...,NaN,Answer exists KB,161216104028927,Connections Error - Uh Oh something went wron...,2017,1,1,1,2017-01-02,Websites : Connections (https://connect.corpta...
4,4,204986,LD400 is not reflecting a temporary adjustment,E-mail,Medium,Tax Preparation,Problem / Error,Platinum,LD400 is not reflecting a temporary adjustment,Compliance,...,US,Answer exists KB,041622907012748,Adjustments do not flow to LD400 workpaper and...,2017,1,1,1,2017-01-03,Compliance : Returns / Reports


In [31]:
df.PPA.nunique()

113

In [83]:
filter = ['Compliance', 'International (US)','Office - Excel Add-In','Provision','Partnership / S Corp / Pass-Through']
df1 =  df[df['Product'].isin(filter)]
df1.head()

,index,Case_Number,Subject,Origin,Priority,Team,Case Type,SLA,Problem Statement,Product,...,Jurisdiction,Resolution,KB #,KB Title,Year,Month,Week,week_num,Date,PPA
0,0,204977,Efile Acceptance,Web,Medium,Tax Preparation,Efile,Platinum,We transmitted a Michigan return still have no...,Compliance,...,US.MI,Answer exists KB,041324813494120,How to understand the Transmission Monitor,2017,1,1,1,2017-01-02,Compliance : Returns / Reports
1,1,204979,Oregon STAT help: Previous Case 203787 Client ...,Web,High,Tax Preparation,Question / How To,Silver,Oregon STAT help: Previous Case 203787 Client ...,Provision,...,US.OR,Customer no longer needs assistance,NaN,NaN,2017,1,1,1,2017-01-02,Provision : State
2,2,204980,Data map incorrectly pulling in values,Web,Medium,Tax Preparation,Question / How To,Silver,Corptax Office Import - Credit Account imports...,Office - Excel Add-In,...,US,Answer exists KB,041414821333274,Corptax® Office - How to place an IF formula i...,2017,1,1,1,2017-01-02,Office - Excel Add-In : Import / Export
4,4,204986,LD400 is not reflecting a temporary adjustment,E-mail,Medium,Tax Preparation,Problem / Error,Platinum,LD400 is not reflecting a temporary adjustment,Compliance,...,US,Answer exists KB,041622907012748,Adjustments do not flow to LD400 workpaper and...,2017,1,1,1,2017-01-03,Compliance : Returns / Reports
6,6,204992,Form 2220 Line 19 dates.,Web,Medium,Tax Preparation,Problem / Error,Platinum,Form 2220 Line 19 dates are repeating. This is...,Compliance,...,US,Answer exists KB,041335414275440,IRS e-Services and Business MeF Maintenance in...,2017,1,1,1,2017-01-03,Compliance : Amounts


In [176]:
all_val = df1.PPA.value_counts()
all_val

Compliance : Returns / Reports                             77888
International (US) : Returns / Reports                     14589
Compliance : Amounts                                        8515
Compliance : Install / Rollover                             7121
Compliance : Configuration / Setup                          5964
Partnership / S Corp / Pass-Through : Returns / Reports     5927
International (US) : Foreign Processing                     5461
Provision : Federal                                         4489
Compliance : Import / Export                                3299
Compliance : Environment Maintenance                        3187
International (US) : Tax Cuts and Jobs Act                  3089
International (US) : Calculation / Posting                  3041
Provision : Configuration / Setup                           2787
Provision : State                                           1901
International (US) : Configuration / Setup                  1673
Compliance : BEPS CbC    

In [85]:
filter2 = list(df1.PPA.value_counts()[df1.PPA.value_counts() < 1600].index)
len(filter2)

39

In [86]:
filter2

['Compliance : BEPS CbC',
 'Office - Excel Add-In : Import / Export',
 'Partnership / S Corp / Pass-Through : K1 Data',
 'Compliance : Printing',
 'Compliance : Backup / Restore',
 'Office - Excel Add-In : Functions',
 'International (US) : Domestic Processing',
 'Office - Excel Add-In : Excel Templates',
 'Partnership / S Corp / Pass-Through : Allocate',
 'Office - Excel Add-In : Data Connections',
 'Partnership / S Corp / Pass-Through : Amounts',
 'Compliance : Tax Cuts and Jobs Act',
 'Provision : Import / Export',
 'Compliance : System Requirements',
 'Partnership / S Corp / Pass-Through : Setup',
 'Compliance : Entity Obligations',
 'International (US) : Import',
 'Partnership / S Corp / Pass-Through : K2/K3',
 'Compliance : Trainer on Call',
 'Office - Excel Add-In : Data Profile',
 'Provision : Tax Contingency (FIN48)',
 'Partnership / S Corp / Pass-Through : Import / Export',
 'Provision : Login',
 'Compliance : Login',
 'Provision : Install',
 'International (US) : Trainer on 

In [87]:
df2 =  df[df['PPA'].isin(filter2)]
df2.head()

,index,Case_Number,Subject,Origin,Priority,Team,Case Type,SLA,Problem Statement,Product,...,Jurisdiction,Resolution,KB #,KB Title,Year,Month,Week,week_num,Date,PPA
2,2,204980,Data map incorrectly pulling in values,Web,Medium,Tax Preparation,Question / How To,Silver,Corptax Office Import - Credit Account imports...,Office - Excel Add-In,...,US,Answer exists KB,041414821333274,Corptax® Office - How to place an IF formula i...,2017,1,1,1,2017-01-02,Office - Excel Add-In : Import / Export
35,35,205047,OECD BEPS CbC Dividend Income,Web,Medium,Tax Preparation,Question / How To,Silver,OECD guidelines and IRS regs indicate that div...,Compliance,...,NaN,Create/Update KB,NaN,NaN,2017,1,1,1,2017-01-03,Compliance : BEPS CbC
55,55,205101,Cannot print the first two pages of the IN Par...,Web,Medium,Tax Preparation,Problem / Error,Silver,Cannot print the first two pages of the IN Par...,Compliance,...,US.IN,Provide solution - no KB required (e.g. client...,NaN,NaN,2017,1,1,1,2017-01-03,Compliance : Printing
58,58,205111,Corptax OFFICE,Web,High,Tax Preparation,Question / How To,Silver,Have question on the need to license Corptax O...,Office - Excel Add-In,...,NaN,Provide solution - no KB required (e.g. client...,NaN,NaN,2017,1,1,1,2017-01-03,Office - Excel Add-In : Functions
107,107,205227,Corptax Office Sample Template 4797,E-mail,Medium,Tax Preparation,Question / How To,Silver,Corptax Office: Is there a sample template ava...,Office - Excel Add-In,...,NaN,Answer exists KB,041320517154121,Corptax® Office - Sample Templates (Best Pract...,2017,1,1,1,2017-01-04,Office - Excel Add-In : Import / Export


In [92]:
monthly_corr.Monthly_value = abs(monthly_corr.Monthly_value)

In [105]:
sorted_corr = monthly_corr.sort_values(by = "Monthly_value", ascending=False)
sorted_corr

,PPA1,PPA2,Monthly_value
0,Compliance : Amounts,Compliance : Amounts,1.000000
825,Compliance : Trainer on Call,Compliance : Trainer on Call,1.000000
1430,Office - Excel Add-In : Data Connections,Office - Excel Add-In : Data Connections,1.000000
1375,International (US) : Trainer on Call,International (US) : Trainer on Call,1.000000
1320,International (US) : Tax Cuts and Jobs Act,International (US) : Tax Cuts and Jobs Act,1.000000
...,...,...,...
2540,Provision : Import / Export,Compliance : Backup / Restore,0.000623
769,Compliance : Tax Cuts and Jobs Act,Compliance : System Requirements,0.000316
716,Compliance : System Requirements,Compliance : Tax Cuts and Jobs Act,0.000316
1074,International (US) : Foreign Processing,Provision : Install,0.000055


In [119]:
filter2

['Compliance : BEPS CbC',
 'Office - Excel Add-In : Import / Export',
 'Partnership / S Corp / Pass-Through : K1 Data',
 'Compliance : Printing',
 'Compliance : Backup / Restore',
 'Office - Excel Add-In : Functions',
 'International (US) : Domestic Processing',
 'Office - Excel Add-In : Excel Templates',
 'Partnership / S Corp / Pass-Through : Allocate',
 'Office - Excel Add-In : Data Connections',
 'Partnership / S Corp / Pass-Through : Amounts',
 'Compliance : Tax Cuts and Jobs Act',
 'Provision : Import / Export',
 'Compliance : System Requirements',
 'Partnership / S Corp / Pass-Through : Setup',
 'Compliance : Entity Obligations',
 'International (US) : Import',
 'Partnership / S Corp / Pass-Through : K2/K3',
 'Compliance : Trainer on Call',
 'Office - Excel Add-In : Data Profile',
 'Provision : Tax Contingency (FIN48)',
 'Partnership / S Corp / Pass-Through : Import / Export',
 'Provision : Login',
 'Compliance : Login',
 'Provision : Install',
 'International (US) : Trainer on 

# for Compliance

In [126]:
[k for k in filter2 if 'Compliance' in k]

['Compliance : BEPS CbC',
 'Compliance : Printing',
 'Compliance : Backup / Restore',
 'Compliance : Tax Cuts and Jobs Act',
 'Compliance : System Requirements',
 'Compliance : Entity Obligations',
 'Compliance : Trainer on Call',
 'Compliance : Login',
 'Compliance : Security Assessment',
 'Compliance : Global Minimum Tax (GMT)']

In [169]:
# if (sorted_corr[sorted_corr['PPA1'].isin([i])][1:3]["Monthly_value"].values > 0.4):

In [161]:
for i in [k for k in filter2 if 'Compliance' in k]:
    if sorted_corr[sorted_corr['PPA1'].isin([i])][1:3]["Monthly_value"].values[0] > 0.7:
        print(i)
        print(sorted_corr[sorted_corr['PPA1'].isin([i])][1:3]["PPA2"].values)
        print(sorted_corr[sorted_corr['PPA1'].isin([i])][1:3]["Monthly_value"].values)
        print("============================")

Compliance : BEPS CbC
['International (US) : Domestic Processing' 'Compliance : Amounts']
[0.72816268 0.56865158]
Compliance : Tax Cuts and Jobs Act
['International (US) : Tax Cuts and Jobs Act'
 'Compliance : Trainer on Call']
[0.79567905 0.61304201]
Compliance : Trainer on Call
['Compliance : Returns / Reports' 'Compliance : Tax Cuts and Jobs Act']
[0.73688284 0.61304201]
Compliance : Login
['Office - Excel Add-In : Login' 'Office - Excel Add-In : Install']
[1. 1.]
Compliance : Security Assessment
['Compliance : Global Minimum Tax (GMT)' 'Provision : Security Assessment']
[0.92493186 0.75557484]
Compliance : Global Minimum Tax (GMT)
['Compliance : Security Assessment' 'Compliance : Login']
[0.92493186 0.70181003]


### Compliance : Global Minimum Tax (GMT) || Compliance : Security Assessment (0.9249)

In [ ]:
df[(df.a != -1) & (df.b != -1)]

In [238]:
temp = df[(df['Product'].isin(["Compliance"])) | (df['PPA'].isin(["Compliance : Security Assessment"]))]
len(temp)

111041

In [230]:
temp = temp[temp['PPA'].isin(["Compliance : Security Assessment"])]
len(temp)

24

In [279]:
# temp_fill = ["Compliance : Global Minimum Tax (GMT)","Compliance : Security Assessment"] #very less
# temp_fill = ['Compliance : Global Minimum Tax (GMT)','Provision : Security Assessment'] #very less
# temp_fill = ['International (US) : Domestic Processing','Compliance : Amounts'] not eligible 
#temp_fill = ['Compliance : Global Minimum Tax (GMT)','Compliance : Security Assessment']
temp_fill = ['Compliance : Amounts','Compliance : Returns / Reports']
temp_fill = ['Compliance : Global Minimum Tax (GMT)','Compliance : Login']
temp_fill = ['Compliance : Returns / Reports','Compliance : Trainer on Call']
temp_fill = ['Compliance : Global Minimum Tax (GMT)','Compliance : Login']
temp_fill = ['Compliance : Tax Cuts and Jobs Act','Compliance : Trainer on Call']
temp_fill = ['International (US) : Calculation / Posting','International (US) : Returns / Reports']
temp_fill = ['International (US) : Foreign Processing','International (US) : Returns / Reports']
temp_fill = ['International (US) : Calculation / Posting','International (US) : Foreign Processing']
temp_fill = ["International (US) : Calculation / Posting",'International (US) : Trainer on Call']
temp_fill = ["Office - Excel Add-In : Data Profile","Office - Excel Add-In : Import / Export"]
temp_fill = ["Partnership / S Corp / Pass-Through : K1 Data","Partnership / S Corp / Pass-Through : Returns / Reports"]
for i in temp_fill:
    print(all_val[i])

1294
5927


In [281]:
temp_df =  df[df['PPA'].isin(temp_fill)]
temp_df.PPA.value_counts().sum()

7221

In [247]:
# temp = df1[df1.PPA == "Compliance : Tax Cuts and Jobs Act"][["Case_Number","Date"]]
# newdf = pd.DataFrame(temp.groupby([temp["Date"].dt.date]).count()["Case_Number"])
# newdf = newdf.reset_index()
# newdf.index = newdf.Date
# newdf['Date'] = pd.to_datetime(newdf['Date'], errors='coerce')
# newdf = newdf.asfreq('D')
# newdf = newdf.groupby(pd.Grouper(freq="M")).sum()
# len(newdf)

In [248]:
temp_df =  df[df['PPA'].isin(temp_fill)]
temp = temp_df
newdf = pd.DataFrame(temp.groupby([temp["Date"].dt.date]).count()["Case_Number"])
newdf = newdf.reset_index()
newdf.index = newdf.Date
newdf['Date'] = pd.to_datetime(newdf['Date'], errors='coerce')
newdf = newdf.asfreq('D')
newdf = newdf.groupby(pd.Grouper(freq="M")).sum()
len(newdf)

68

In [225]:
newdf.sum()

Case_Number    26.0
dtype: float64

In [205]:
print("length of dataframe : ",newdf.shape[0])
f = Forecaster(y=newdf['Case_Number'],current_dates=newdf.index)
f.generate_future_dates(12)
f.set_test_length(.2)
critical_pval = 0.05
stat, pval, _, _, _, _ = f.adf_test(full_res=True)
print('the series is {}'.format('stationary' if pval < critical_pval else 'not stationary'))
print("Lets Make Data Stationary First")
f.set_estimator('mlr')
f.add_ar_terms(12) 
f.add_seasonal_regressors('quarter',dummy=True) # 2.
f.add_time_trend() # 2.
f.diff()
stat, pval, _, _, _, _ = f.adf_test(full_res=True)
print('Now, This series is {}'.format('stationary' if pval < critical_pval else 'not stationary'))
f.manual_forecast()
results = f.export(to_excel=True,
           excel_name='mlr.xlsx')
print(results["model_summaries"]["InSampleRMSE"][0],2)
print(round(results["model_summaries"]["TestSetRMSE"][0],2))

length of dataframe :  68
the series is not stationary
Lets Make Data Stationary First
Now, This series is stationary
295.7643107308625 2
259.16


In [201]:
# res = {}
# filter_list = list(df1.PPA.value_counts()[df1.PPA.value_counts() > 1600].index)
# filter_list = temp_fill
temp_df =  df[df['PPA'].isin(temp_fill)]
train_rmse = []
test_rmse = []
months = []
    
#grouping by month
temp = temp_df
newdf = pd.DataFrame(temp.groupby([temp["Date"].dt.date]).count()["Case_Number"])
newdf = newdf.reset_index()
newdf.index = newdf.Date
newdf['Date'] = pd.to_datetime(newdf['Date'], errors='coerce')
newdf = newdf.asfreq('D')
newdf = newdf.groupby(pd.Grouper(freq="M")).sum()

print("length of dataframe : ",newdf.shape[0])
f = Forecaster(y=newdf['Case_Number'],current_dates=newdf.index)
f.generate_future_dates(12)
f.set_test_length(.2)
critical_pval = 0.05
stat, pval, _, _, _, _ = f.adf_test(full_res=True)
print('the series is {}'.format('stationary' if pval < critical_pval else 'not stationary'))
    if pval < critical_pval:
        auto_arima(
            f,
            start_P=1,
            start_q=1,
            max_p=10,
            max_q=10,
            m=12,
            seasonal=True,
            max_P=10, 
            max_D=10,
            max_Q=10,
            max_d=10,
            trace=True,
            error_action='ignore',
            suppress_warnings=True,
            stepwise=True,
            information_criterion="aic",
            alpha=0.05,
            scoring='mse',
            call_me='auto_arima',)
        f.plot_test_set(ci=False,models='auto_arima')
        plt.title('ARIMA Test-Set Performance',size=14)
        plt.show()
        results = f.export(to_excel=True,excel_name='mlr.xlsx')
        train_rmse.append(round(results["model_summaries"]["InSampleRMSE"][0],2))
        test_rmse.append(round(results["model_summaries"]["TestSetRMSE"][0],2))
        
        for j in range(0,4):
            months.append(round(results["lvl_fcsts"].iloc[j]["auto_arima"]))
        model = "arima-S"
        res[i] = [model, train_rmse[0], test_rmse[0], newdf.min()[0], newdf.max()[0], newdf.mean()[0], newdf.std()[0],months[0],months[1],months[2],months[3]]

    else:
        print("Lets Make Data Stationary First")
        f.set_estimator('mlr')
        f.add_ar_terms(12) 
        f.add_seasonal_regressors('quarter',dummy=True) # 2.
        f.add_time_trend() # 2.
        f.diff()
        stat, pval, _, _, _, _ = f.adf_test(full_res=True)
        print('Now, This series is {}'.format('stationary' if pval < critical_pval else 'not stationary'))
        f.manual_forecast()
        results = f.export(to_excel=True,
                   excel_name='mlr.xlsx')
        train_rmse.append(round(results["model_summaries"]["InSampleRMSE"][0],2))
        test_rmse.append(round(results["model_summaries"]["TestSetRMSE"][0],2))
        
        for j in range(0,4):
            months.append(round(results["lvl_fcsts"].iloc[j]["mlr"]))
            
        model = "mlr-NS"
        res[i] = [model, train_rmse[0], test_rmse[0], newdf.min()[0], newdf.max()[0], newdf.mean()[0], newdf.std()[0],months[0],months[1],months[2],months[3]]
    print("============================END=================================")


Compliance : Returns / Reports
length of dataframe :  68
the series is not stationary
Lets Make Data Stationary First
Now, This series is stationary
============================END=================================
Compliance : Tax Cuts and Jobs Act
length of dataframe :  54
the series is not stationary
Lets Make Data Stationary First
Now, This series is stationary
============================END=================================


In [318]:
ind = ["model","train_rmse","test_rmse","min", "max", "mean", "std", "sept", "oct", "nov", "dec"]
res_df = pd.DataFrame(res)
res_df.index = ind
round(res_df,2).T

,model,train_rmse,test_rmse,min,max,mean,std,sept,oct,nov,dec
Compliance : Returns / Reports,mlr-NS,293.46,254.61,456.0,4449.0,1145.411765,799.908021,1496,2107,1006,368
International (US) : Returns / Reports,mlr-NS,56.09,69.47,44.0,684.0,214.544118,149.335387,489,241,99,68
Compliance : Amounts,mlr-NS,20.93,28.7,33.0,352.0,125.220588,76.434704,208,234,134,61
Compliance : Install / Rollover,mlr-NS,28.99,54.25,30.0,280.0,104.720588,67.240012,78,75,42,37
Compliance : Configuration / Setup,mlr-NS,13.82,10.73,42.0,164.0,87.705882,26.948851,74,44,30,17
Partnership / S Corp / Pass-Through : Returns / Reports,mlr-NS,24.78,33.62,6.0,271.0,87.161765,64.723663,215,40,31,13
International (US) : Foreign Processing,arima-S,32.71,32.03,19.0,220.0,80.308824,44.954789,97,67,61,53
Provision : Federal,mlr-NS,13.98,12.44,24.0,247.0,66.014706,36.106597,51,50,44,37
Compliance : Import / Export,mlr-NS,9.0,10.06,11.0,110.0,48.514706,22.153126,32,9,7,3
Compliance : Environment Maintenance,arima-S,47.74,24.59,14.0,393.0,46.867647,50.457875,38,36,33,41


In [291]:
res_df

,Compliance : Returns / Reports,International (US) : Returns / Reports,Compliance : Amounts,Compliance : Install / Rollover,Compliance : Configuration / Setup,International (US) : Foreign Processing,Provision : Federal,Compliance : Import / Export,Compliance : Environment Maintenance,International (US) : Tax Cuts and Jobs Act,International (US) : Calculation / Posting,Provision : Configuration / Setup,Provision : State,International (US) : Configuration / Setup
model,mlr,mlr,mlr,mlr,mlr,arima-S,mlr,mlr,arima-S,mlr,arima-S,arima-S,mlr,arima-S
train_rmse,293.46,56.09,20.93,28.99,13.82,32.71,13.98,9.0,47.74,12.39,21.69,11.99,6.95,8.14
test_rmse,254.61,69.47,28.7,54.25,10.73,32.03,12.44,10.06,24.59,13.23,12.03,9.78,10.28,10.48
min,456.0,44.0,33.0,30.0,42.0,19.0,24.0,11.0,14.0,0.0,7.0,18.0,5.0,7.0
max,4449.0,684.0,352.0,280.0,164.0,220.0,247.0,110.0,393.0,334.0,166.0,100.0,79.0,52.0
mean,1145.411765,214.544118,125.220588,104.720588,87.705882,80.308824,66.014706,48.514706,46.867647,57.203704,44.720588,40.985294,27.955882,24.602941
std,799.908021,149.335387,76.434704,67.240012,26.948851,44.954789,36.106597,22.153126,50.457875,72.929374,31.394859,16.28798,15.411213,10.16159
sept,1496,489,208,78,74,97,51,32,38,18,78,32,30,27
oct,2107,241,234,75,44,67,50,9,36,18,37,31,33,20
nov,1006,99,134,42,30,61,44,7,33,16,21,30,27,14
